In [1]:
from src.config import Config
config = Config()

### Generate Training Data and Visualize

In [2]:
import matplotlib.pyplot as plt 
from src.data.data_simulator import generate_data

train_data = generate_data(n_periods=config.training_period, mode="train", seed=42)
plt.figure(figsize=(12, 8))

# Plot Demand
plt.subplot(3, 1, 1)
plt.plot(train_data['time'], train_data['demand'], label='Demand', color='blue')
plt.title('Training Data: Demand over Time')
plt.xlabel('Time')
plt.ylabel('Demand')
plt.legend()

# Plot Lead Time
plt.subplot(3, 1, 2)
plt.plot(train_data['time'], train_data['lead_time'], label='Lead Time', color='green')
plt.title('Training Data: Lead Time over Time')
plt.xlabel('Time')
plt.ylabel('Lead Time')
plt.legend()

plt.tight_layout()
plt.show()

### Train the RL agent using PPO

In [3]:
from src.reinforcement_learning import train_ppo_model
train_ppo_model(train_data)

Output()

### Generate Test Data and Visualize

In [4]:
# Generate test data and evaluate performance
test_data = generate_data(n_periods=config.test_period, mode="test", seed=24)

plt.figure(figsize=(12, 8))

# Plot Demand
plt.subplot(3, 1, 1)
plt.plot(test_data['time'], test_data['demand'], label='Demand', color='blue')
plt.title('Training Data: Demand over Time')
plt.xlabel('Time')
plt.ylabel('Demand')
plt.legend()

# Plot Lead Time
plt.subplot(3, 1, 2)
plt.plot(test_data['time'], test_data['lead_time'], label='Lead Time', color='green')
plt.title('Training Data: Lead Time over Time')
plt.xlabel('Time')
plt.ylabel('Lead Time')
plt.legend()

plt.tight_layout()
plt.show()

### Load the trained RL model and Compare Performances with Pure RL and Pure RO

In [5]:
from src.performance_evaluation import PerformanceEvaluator
from src.robust_optimization import robust_order_quantity
from src.inventory_env import InventoryEnv
from stable_baselines3 import PPO

# Load the trained RL model
rl_model = PPO.load(config.model_path)

evaluator = PerformanceEvaluator(
    test_data=test_data,
    env_class=InventoryEnv,
    robust_optimizer=robust_order_quantity,
    rl_model=rl_model,
)

performance_table = evaluator.evaluate_all_methods()
print("\nPerformance Evaluation:")
print(performance_table)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-25

Performance Evaluation:
            Total Cost  Average Cost per Period  Stockout Frequency  \
pure_rl  212581.137394               212.581137               335.0   
pure_ro  569917.086066               569.917086               950.0   
RLeRO    237555.669738               237.555670               410.0   

         Average Inventory Level  Inventory Turnover  \
pure_rl                22.763098         2190.162842   
pure_ro               -92.804909                 NaN   
RLeRO                   9.307097         5356.652832   

         Robust Decisions Triggered  
pure_rl                         0.0  
pure_ro                         0.0  
RLeRO                          96.0  
